## Verify Original Status of Colab
The cell outputs in this section is for reference.

In [1]:
!python -V

Python 3.11.11


In [2]:
!pip list | grep torch

torch                              2.5.1+cu124
torchaudio                         2.5.1+cu124
torchsummary                       1.5.1
torchvision                        0.20.1+cu124


### Known Issues of Colab
No need to run this section, which is only for note-taking.

#### 1. Inconsistent `nvidia` Packages

In [3]:
!pip check

ipython 7.34.0 requires jedi, which is not installed.
pygobject 3.42.1 requires pycairo, which is not installed.
torch 2.5.1+cu124 has requirement nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2.
torch 2.5.1+cu124 has requirement nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82.
torch 2.5.1+cu124 has requirement nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82.
torch 2.5.1+cu124 has requirement nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-runtime-cu12 12.5.82.
torch 2.5.1+cu124 has requirement nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75.
torch 2.5.1+cu124 has require

They will be fixed by reinstallation below.

In [ ]:
!pip uninstall -y torch torchaudio torchsummary torchvision

In [ ]:
!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

In [ ]:
!pip install jedi

It seems impossible to fix pygobject...

In [ ]:
!pip uninstall -y pygobject

In [8]:
!pip check

No broken requirements found.


#### 2. Mismatch and Unknown Linux Headers
It prevents `dkms` from updating display drivers; seems no way to be resolved.

In [9]:
!uname -r  # 6.1.85+ is unknown
!ls /usr/src  # 5.15.0 is common for Ubuntu 22.04

6.1.85+
linux-headers-5.15.0-133  linux-headers-5.15.0-133-generic  python3.10


## Work with Cohore Repo

The hard-coded repo and branch below should be changed in the future.

**One should also not clone from branch after each merge of PR.**

In [ ]:
%cd /content
!rm -rf s4
!git clone -b build/deps-py312 https://github.com/cataluna84/s4.git
#!git clone https://github.com/cataluna84/s4.git

In [ ]:
%cd /content/s4

## Install a Newer Python in Colab's Ubuntu 22.04

The command below shows that Colab's Ubuntu has already included `ppa:deadsnakes/ppa`.

In [12]:
!apt-cache search -n "^python3.1[234]-?"

python3.12 - Interactive high-level object-oriented language (version 3.12)
python3.12-dbg - Debug Build of the Python Interpreter (version 3.12)
python3.12-dev - Header files and a static library for Python (v3.12)
python3.12-examples - Examples for the Python language (v3.12)
python3.12-full - Python Interpreter with complete class library (version 3.12)
python3.12-gdbm - GNU dbm database support for Python (version 3.12)
python3.12-gdbm-dbg - GNU dbm database support for Python (version 3.12 debug extension)
python3.12-lib2to3 - lib2to3 package for Python (version 3.12)
python3.12-tk - Tkinter - Writing Tk applications with Python (version 3.12)
python3.12-tk-dbg - Tkinter - Writing Tk applications with Python (version 3.12 debug extension)
python3.12-venv - Interactive high-level object-oriented language (pyvenv binary, version 3.12)
python3.13 - Interactive high-level object-oriented language (version 3.13)
python3.13-dbg - Debug Build of the Python Interpreter (version 3.13)
pyth

### Install Python 3.12

In [ ]:
!make py312

In [14]:
!python -V

Python 3.12.9


## Prepare a Virtual Environment

In [15]:
!python -m venv ~/venv/py312_cu126_pt260

In [16]:
!source ~/venv/py312_cu126_pt260/bin/activate; python -V; pip list

Python 3.12.9
Package Version
------- -------
pip     24.3.1


## Update Dependencies

In [ ]:
!source ~/venv/py312_cu126_pt260/bin/activate; make pt260

In [18]:
!source ~/venv/py312_cu126_pt260/bin/activate; pip list | grep torch

torch                    2.6.0+cu126
torchaudio               2.6.0+cu126
torchvision              0.21.0+cu126


In [ ]:
!source ~/venv/py312_cu126_pt260/bin/activate; make lib

In [23]:
!source ~/venv/py312_cu126_pt260/bin/activate; pip check

No broken requirements found.


The venv needs `matplotlib-inline`.

In [ ]:
!source ~/venv/py312_cu126_pt260/bin/activate; pip install matplotlib-inline

## Train S4 with IMDB

Remember to put your own W&B API key to Colab's Secrets.

In [24]:
from google.colab import userdata


wandb_api_key = userdata.get("WANDB_API_KEY")

In [ ]:
%env WANDB_API_KEY={wandb_api_key}

The values of `n_layers` and `d_model` are for smaller memory footprints.

In [ ]:
!source ~/venv/py312_cu126_pt260/bin/activate; python -m train pipeline=imdb model=s4 dataset.val_split=0.1 model.n_layers=3 model.d_model=128